In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
from geopy import Nominatim
import base64
from IPython.html.widgets import interact #Needed for interactive graph
#from pygeocoder import geocoder
import folium
import json

/Users/adriaan/anaconda/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


We kindly request the reader to put the URL in https://nbviewer.jupyter.org/ to view te graphs.

# Europe

The data was extracted from http://ec.europa.eu/eurostat/data/database. 

In [2]:
df=pd.DataFrame.from_csv('tsdec450.tsv', sep='\t', header=0)
df.head()

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
"age,unit,sex,geo\time",,,,,,,,,,,,,,,,,,,,,
"TOTAL,PC_ACT,T,AT",:,:,:,:,4.3,4.2,4.7,4.7,4.7,4.2,...,4.9,4.1,5.3,4.8,4.6,4.9,5.4,5.6,5.7,6.0
"TOTAL,PC_ACT,T,BE",6.6,6.4,7.1,8.6,9.8,9.7,9.5,9.2,9.3,8.4,...,7.5,7.0,7.9,8.3,7.2,7.6,8.4,8.5,8.5,7.8
"TOTAL,PC_ACT,T,BG",:,:,:,:,:,:,:,:,:,:,...,6.9,5.6,6.8,10.3 i,11.3,12.3,13.0,11.4,9.2,7.6
"TOTAL,PC_ACT,T,CY",:,:,:,:,:,:,:,:,:,:,...,3.9,3.7,5.4,6.3,7.9,11.9,15.9,16.1,15.0,13.0
"TOTAL,PC_ACT,T,CZ",:,:,:,4.3,4.3,4.0,3.9,4.8,6.5,8.7,...,5.3,4.4,6.7,7.3,6.7,7.0,7.0,6.1,5.1,4.0


In [3]:
#Create multi index for easy processing.
df["age"] = np.nan
df["unit"]=np.nan
df["sex"]=np.nan #We select total afterwards
df["geo/time"]=np.nan
for i,indexStr in enumerate(df.index):
    stringList=indexStr.split(',')
    df.iloc[i, df.columns.get_loc('age')] = stringList[0]
    df.iloc[i,df.columns.get_loc('unit')]=stringList[1]
    df.iloc[i,df.columns.get_loc('sex')]=stringList[2]
    df.iloc[i,df.columns.get_loc('geo/time')]=stringList[3]
                
df=df.set_index(['age', 'unit', 'sex','geo/time'])
#Now select the needed latest data 


Now we extract the data we need. And do some minor clean up operations.

In [4]:
#We want the data of the total age and the percentage of the active population of 2016
countryDf=df.loc[('TOTAL', 'PC_ACT', 'T')]["2016 "].copy()

#Dropping columns that do not correspond to a country
countryDf.drop(['EA18','EA19','EU27','EU28'],inplace=True)

#Greece is abbreviated as EL 
nameIndexes=[]
for countryCode in countryDf.index:
    if countryCode =="EL":
        countryCode="GR"
    if countryCode =="UK":
        countryCode="GB"    
    #nameIndexes.append(pycountry.countries.get(alpha_2=countryCode).name)
    nameIndexes.append(countryCode)
countrySeries=pd.Series(countryDf.values,nameIndexes)
countryDf=pd.Series.to_frame(countrySeries)
countryDf.reset_index(inplace=True)
countryDf.columns = ['id', 'Unemployment']
countryDf.head()

,id,Unemployment
0,AT,6.0
1,BE,7.8
2,BG,7.6
3,CY,13.0
4,CZ,4.0


In [5]:
cuttingPercentages=pd.qcut(countryDf["Unemployment"],5, labels=False,retbins=True, precision=1, duplicates='raise')[-1] 
#Using a qcut to create intervals with equal number of  elements this will be used throughout the notebook.

In [6]:
#Choropleth
m = folium.Map(location=[46.519962, 6.633597], zoom_start=3)
topo_json_data = json.load(open('europe.topojson.json'))
m.choropleth(
    geo_data=topo_json_data,
    name='choropleth',
    data=countryDf,
    columns=['id', 'Unemployment'],
    key_on='feature.id',
    threshold_scale = list(cuttingPercentages),
    fill_color= 'GnBu',
    fill_opacity=0.9,
    line_opacity=0.4,
    legend_name='Unemployment Rate (%)',
    topojson = 'objects.europe'
)
#m.save("europe.html")
#m

You can see the map http://htmlpreview.github.com/?https://github.com/adriaanvangervenEPFL/testrepoada/blob/master/homework3/europe.html

# Switzerland

<b>2)Per canton</b>

The data of the swiss cantons is collected from amstat.ch. We used the "taux de chômage" sorted by canton to get our data.

In [7]:
dfCanton=pd.read_excel('rateSuisseCantonsGerman.xlsx')
dfCanton.head()

,Kanton,Monat,Oktober 2016,November 2016,Dezember 2016,Januar 2017,Februar 2017,März 2017,April 2017,Mai 2017,Juni 2017,Juli 2017,August 2017,September 2017,Gesamt
0,NaN,Metriken,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote
1,Zürich,NaN,3.6,3.7,3.8,3.9,3.9,3.8,3.6,3.5,3.4,3.4,3.4,3.3,3.6
2,Bern,NaN,2.6,2.8,3,3,3,2.9,2.7,2.6,2.4,2.4,2.5,2.4,2.7
3,Luzern,NaN,1.9,2,2.2,2.2,2.2,2,1.9,1.8,1.7,1.7,1.8,1.7,1.9
4,Uri,NaN,1,1.1,1.3,1.5,1.4,1.3,1.2,1,0.8,0.7,0.6,0.6,1.1


We filter trough the data frame. Working away inconsistencies arround the naming and selecting the average of the last year.

In [8]:
def adressMapper(canton):
    """
    Maps the adress to a more commonly used name.
    """
    address, (latitude, longitude) = geolocator.geocode(canton+" Schweiz")
    return address.split()[0][:-1]

In [9]:
dfCanton=pd.read_excel('rateSuisseCantonsGerman.xlsx')
dfCanton.head()
dfCanton=dfCanton[['Kanton','Gesamt']].iloc[1:-1]#remove first and last row and select the columns we nead.
#Since each canton has an unique name we can index by canton
dfCanton.set_index('Kanton',drop=True,inplace=True,verify_integrity=True)
dfCanton['Gesamt']=dfCanton['Gesamt'].astype(float) #Convert to float
dfCanton.reset_index(inplace=True)
dfCanton.columns = ['name', 'Unemployment']
dfCanton.head()

,name,Unemployment
0,Zürich,3.6
1,Bern,2.7
2,Luzern,1.9
3,Uri,1.1
4,Schwyz,1.8


In [10]:
"""
#Generating from the mapping dict left in for information.
mappingDict={}
for index,name in enumerate(dfCanton["name"]):
    mappingDict[name]=adressMapper(name)
"""

mappingDict={'Zürich': 'Zürich', 'Bern': 'Bern', 'Luzern': 'Luzern', 'Uri': 'Uri', 'Schwyz': 'Schwyz', 'Obwalden': 'Obwalden', 'Nidwalden': 'Nidwalden', 'Glarus': 'Glarus', 'Zug': 'Zug', 'Freiburg': 'Fribourg/Freiburg', 'Solothurn': 'Solothurn', 'Basel-Stadt': 'Basel-Stadt', 'Basel-Landschaft': 'Basel-Landschaft', 'Schaffhausen': 'Schaffhausen', 'Appenzell Ausserrhoden': 'Appenzel', 'Appenzell Innerrhoden': 'Appenzel', 'St. Gallen': 'St', 'Graubünden': 'Graubünden/Grigioni/Grischun', 'Aargau': 'Aargau', 'Thurgau': 'Thurgau', 'Tessin': 'Ticino', 'Waadt': 'Vaud', 'Wallis': 'Valais/Wallis', 'Neuenburg': 'Neuchâtel', 'Genf': 'Genève', 'Jura': 'Jura'}
#Replace some manually
mappingDict['Appenzell Ausserrhoden']='Appenzell Ausserrhoden'
mappingDict['Appenzell Innerrhoden']='Appenzell Innerrhoden'
mappingDict['Freiburg']= 'Fribourg'
mappingDict['Graubünden']='Graubünden/Grigioni'
mappingDict['St. Gallen']='St. Gallen'
mappingDict['Bern']='Bern/Berne'


In [11]:
for index,name in enumerate(dfCanton["name"]):
    #We replace the names by the names used in the topojson
    dfCanton.iloc[index, dfCanton.columns.get_loc('name')] = mappingDict[name]
dfCanton.head()

,name,Unemployment
0,Zürich,3.6
1,Bern/Berne,2.7
2,Luzern,1.9
3,Uri,1.1
4,Schwyz,1.8


In [12]:
#Choropleth2
beginLocation=[46.911492, 8.090388]
beginZoom=7
s = folium.Map(location=beginLocation, zoom_start=beginZoom)
topo_json_data = json.load(open('ch-cantons.topojson.json'))
colorscheme=pd.qcut(dfCanton["Unemployment"],5, labels=False,retbins=True, precision=1, duplicates='raise')[-1]

In [13]:
s.choropleth(
    geo_data=topo_json_data,
    name='choropleth',
    data=dfCanton,
    columns=['name', 'Unemployment'],
    key_on='feature.properties.name',
    threshold_scale = list(colorscheme),
    fill_color= 'GnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment Rate (%)',
    topojson ='objects.cantons'
)
#folium.LayerControl().add_to(m)

s.save('mapsuisse1.html')
s


U can see the map by using the notebook viewer: 


<b>3) Include the information about suisse and foreign workers</b>

In [14]:
dfCantonForeign=pd.read_excel('rateSuisseCantonForeignersGerman.xlsx')
dfCantonForeign.head()
#Same process as before but with hierachal index
dfCantonForeign=dfCantonForeign[['Kanton','Nationalität','Gesamt']].iloc[1:-1]#Also leave out last column
dfCantonForeign.columns = ['Canton', 'Nationality','Unemployement']
dfCantonForeign["Unemployement"]=dfCantonForeign["Unemployement"].astype(float)
for index,name in enumerate(dfCantonForeign["Canton"]):
    dfCantonForeign.iloc[index, dfCantonForeign.columns.get_loc('Canton')] = mappingDict[name]
dfCantonForeign.head()

,Canton,Nationality,Unemployement
1,Zürich,Ausländer,5.9
2,Zürich,Schweizer,2.7
3,Bern/Berne,Ausländer,6.5
4,Bern/Berne,Schweizer,2.0
5,Luzern,Ausländer,4.5


In [15]:
#We have to show the difference between the two percentages
seriesCantonForeingSuisse=pd.Series()
for canton,cantonDf in dfCantonForeign.groupby("Canton"):
    unEmployementForeign=cantonDf.loc[cantonDf['Nationality'] == "Ausländer"].Unemployement.values[0]
    unEmployementSwiss=cantonDf.loc[cantonDf['Nationality'] == "Schweizer"].Unemployement.values[0]
    difference=unEmployementForeign-unEmployementSwiss
    seriesCantonForeingSuisse[canton]=difference
dfCantonForeign=seriesCantonForeingSuisse.to_frame()
dfCantonForeign.reset_index(inplace=True)
dfCantonForeign.columns=["Canton","unemployementDifference"]


In [16]:
#visualize it the same way as before
s = folium.Map(location=beginLocation, zoom_start=beginZoom)
topo_json_data = json.load(open('ch-cantons.topojson.json'))
colorscheme=pd.qcut(dfCantonForeign["unemployementDifference"],5, labels=False,retbins=True, precision=1, duplicates='raise')[-1]
s.choropleth(
    geo_data=topo_json_data,
    name='choropleth',
    data=dfCantonForeign,
    columns=['Canton', 'unemployementDifference'],
    key_on='feature.properties.name',
    threshold_scale = list(colorscheme),
    fill_color= 'GnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Unemployment difference between foreigners-suisse (%)',
    topojson ='objects.cantons'
)

#folium.LayerControl().add_to(m)
s.save("mapsuisse2")
s

U can see the map <a href=mapsuisse2.html>here</a>

4) Now also with age categories

In [17]:
dftem=pd.read_excel('rateSuisseCantonAgeGerman.xlsx')
dftem.head(4)

,Kanton,Altersklassen in 5-Jahresschritten,Monat,Oktober 2016,November 2016,Dezember 2016,Januar 2017,Februar 2017,März 2017,April 2017,Mai 2017,Juni 2017,Juli 2017,August 2017,September 2017,Gesamt
0,NaN,NaN,Metriken,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote,Arbeitslosenquote
1,Zürich,15-19 Jahre,NaN,4,3.8,3.6,3.5,3.3,3,2.7,2.5,2.4,4,4.2,3.9,3.4
2,Zürich,20-24 Jahre,NaN,4.3,4.2,4.3,4.5,4.3,4.1,3.7,3.5,3.4,3.4,3.6,3.7,3.9
3,Zürich,25-29 Jahre,NaN,4.2,4.3,4.5,4.6,4.4,4.2,4,3.9,3.7,3.6,3.6,3.5,4


In [18]:
#Same
dfCantonAge=pd.read_excel('rateSuisseCantonAgeGerman.xlsx')
dfCantonAge.columns.values[1] = 'Agegroup'
dfCantonAge=dfCantonAge[['Kanton','Agegroup','Gesamt']].iloc[1:-1]
for index,agegroup in enumerate(dfCantonAge["Agegroup"]):
    #Remove the ans
    cleaned=str(agegroup).split()[0]
    if cleaned=="60":
        cleaned=cleaned+"+"
    dfCantonAge.iloc[index, dfCantonAge.columns.get_loc('Agegroup')]=cleaned
#In the datarame the unkown values are replaced with ... 
#We replace the ... with NaN
dfCantonAge.replace("...",np.nan,inplace=True)
dfCantonAge["Gesamt"]=dfCantonAge["Gesamt"].astype(float)
dfCantonAge.columns=["Canton","Agegroup","Unemployement"]
for index,name in enumerate(dfCantonAge["Canton"]):
    dfCantonAge.iloc[index, dfCantonAge.columns.get_loc('Canton')] = mappingDict[name]
dfCantonAge.head(15)


,Canton,Agegroup,Unemployement
1,Zürich,15-19,3.4
2,Zürich,20-24,3.9
3,Zürich,25-29,4.0
4,Zürich,30-34,4.1
5,Zürich,35-39,4.1
6,Zürich,40-44,3.4
7,Zürich,45-49,3.2
8,Zürich,50-54,3.4
9,Zürich,55-59,3.3
10,Zürich,60+,2.6


We are making barcharts for each canton by age. The barcharts will have a specific color. 
This color represents how the agegroup of the canton is doing compared to the same age group of different cantons. 

In [19]:
#The goal is to get a dictionairy {district{agegroup{color{}}}}
colorDict={}
colors=sns.color_palette("GnBu",5)
agegroups=[]
percentageCutDict={}
for agegroup,ageGroupDf in dfCantonAge.groupby("Agegroup"):
    agegroups.append(agegroup)
    #Find the cuting percentages of this agegroup.
    percentageCuts=pd.qcut(ageGroupDf["Unemployement"],5, labels=False,retbins=True, precision=1, duplicates='raise')[-1]
    percentageCutDict[agegroup]=percentageCuts
    for canton,cantonageDf in ageGroupDf.groupby("Canton"):
        value=cantonageDf["Unemployement"].values[0]
        #Categorise this value in colors using the cutting percentages.
        classified=False
        for index,percentage in enumerate(percentageCuts):
            if index<len(percentageCuts)-1:
                if percentageCuts[index]<=value:
                    if value<percentageCuts[index+1]:
                        color=colors[index]
                        if canton in colorDict:
                            colorDict[canton][agegroup]=color
                        else:
                            colorDict[canton]={agegroup:color}
                        classified=True
            if classified==True:
                break
        if classified==False:
            if not np.isnan(value):
                assert(value==percentageCuts[-1])
                color=colors[-1]
                if canton in colorDict:
                    colorDict[canton][agegroup]=color
                else:
                    colorDict[canton]={agegroup:color}

                
            
            


Now we have a dictionairy which tells us which color we should use for which canton and age group. We use this information to generate the bar plots of each agegroup. Again the colors represent how the canton's rate is compared to the same agegroup for Switzerland as a whole.

In [22]:
import os
if not os.path.exists("figures"):
    os.makedirs("figures")
for canton,ageDf in dfCantonAge.groupby("Canton"):
    plt.close()
    sns.set_style("whitegrid")
    plt.xlabel("age group")
    plt.ylabel("unemployement")
    colorPallete=[colorDict[canton][agegroup] for agegroup in agegroups if agegroup in colorDict[canton]]
    ax=sns.barplot(x="Agegroup", y="Unemployement", data=ageDf.reset_index(),palette=colorPallete)
    ax.set(xlabel='age groups', ylabel='percentage',title="Unemployement rate vs age in: "+canton)
    #Save the barplots for later (remove the / because of directory problems)
    if "/" in canton:
        canton=canton.replace("/","_")
    plt.savefig("figures/"+canton+".png")


In [24]:
#Get the locations of the canton with the help of geolocator.
locations={}
geolocator=Nominatim()
for canton,ageDf in dfCantonAge.groupby("Canton"):
    location = geolocator.geocode("Schweiz " + canton )
    if location==None:
        location=geolocator.geocode("Switzerland"+canton)
    locations[canton]=location
    

In [25]:
def create_map (age):
    """
    Function to create the map for a specific agegroup.
    """
    resolution, width, height = 75, 5, 2
    colorscheme=percentageCutDict[age]
    s = folium.Map(location=[46.941500, 8.398005], zoom_start=7.5)
    topo_json_data = json.load(open('ch-cantons.topojson.json'))
    s.choropleth(
    geo_data=topo_json_data,
    name='choropleth',
    data=dfCanton,
    columns=['name', 'Unemployment'],
    key_on='feature.properties.name',
    threshold_scale = list(colorscheme),
    fill_color= 'GnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Overall unemployement rate for age group: '+str(age),
    topojson ='objects.cantons'
    )
    
    for canton,ageDf in dfCantonAge.groupby("Canton"):
        #Insert the image in a html iframe.
        html = '<img src="data:image/png;base64,{}" style="width: 100%; height: 100%" />'.format
        cantonName=canton
        if "/" in canton:
            cantonName=cantonName.replace("/","_")
            
        encoded =  base64.b64encode(open("figures/"+cantonName+".png", 'rb').read()).decode()


        iframe = folium.IFrame(html(encoded), width=(width*resolution)+20, height=(height*resolution)+20)
        popup = folium.Popup(iframe, max_width=2650)
        icon = folium.Icon(color="blue",icon_color="white",icon="info-sign")
        marker = folium.Marker(location=[locations[canton].latitude, locations[canton].longitude], popup=popup, icon=icon)
        s.add_child(marker);
    return s

Now we can create our final masterpiece. An interactive graph with info buttons to see the specific distribution of the unemployement rate. 
Note that when changing from agegroup both the colors and the scale changes.
Also note that the color on the map is for one agecategory is the same as the color on the chart for that category. 

In [26]:
s=interact(create_map,age=agegroups)
s

<function __main__.create_map>

I'll put in a screenshot of the map to give an idea on how it looks in case you don't want to use the notebook viewer :
 <img src="mapscreenshot.png">
